In [22]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.filters import threshold_local
from PIL import Image

In [54]:
import os
import cv2
import numpy as np

def process_image(image_path, output_folder):
    # Read the image
    original_image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding
    adaptive_thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 25, 4)

    # Use morphological operations to remove noise and enhance lines
    kernel = np.ones((5, 5), np.uint8)
    opening = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)

    # Detect lines using Probabilistic Hough Transform
    lines = cv2.HoughLinesP(opening, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    # Initialize variables to store the longest vertical line
    max_length = 0
    longest_line = None
    if lines is None:
        print(image_path + "fail to rotate")
    else:

        # Find the longest vertical line
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            if (-60 > angle >= -90) or (40 < angle <= 90):  # Consider lines with angle between 80 and 100 degrees as vertical
                if length > max_length:
                    max_length = length
                    longest_line = line

        if longest_line is not None:
            angle_from_vertical = np.arctan2(longest_line[0][3] - longest_line[0][1], longest_line[0][2] - longest_line[0][0]) * 180 / np.pi
            x1, y1, x2, y2 = longest_line[0]
            (h, w) = original_image.shape[:2]
            center = (w // 2, h // 2)

            if(angle_from_vertical < 0):
                rotation_matrix = cv2.getRotationMatrix2D(center, 90+angle_from_vertical, 1.0)  # Negative angle to rotate clockwise
            else:
                rotation_matrix = cv2.getRotationMatrix2D(center, angle_from_vertical-90, 1.0)  # Negative angle to rotate clockwise
            rotated_image = cv2.warpAffine(original_image, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC)# , borderMode=cv2.BORDER_REPLICATE
            
            # Save the rotated image with the original filename
            filename = os.path.basename(image_path)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, rotated_image)

            # Print the angle from the vertical line
            #print(f"Angle from vertical ({filename}): {angle_from_vertical}")

# Folder containing the images
input_folder = "/Users/local_admin/Desktop/thesis/output_model/output_model_synth_cut"
output_folder = "/Users/local_admin/Desktop/thesis/images/synth_cut"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".png"):
        # Process each image
        image_path = os.path.join(input_folder, filename)
        process_image(image_path, output_folder)

print("Processing complete. Rotated images saved in:", output_folder)

Processing complete. Rotated images saved in: /Users/local_admin/Desktop/thesis/images/synth_cut
